In [1]:
from llama_cpp import Llama
import llama_cpp
from datasets import load_dataset
import evaluate
import requests
import json
from tqdm.notebook import tqdm  # Use tqdm.notebook for Jupyter Notebook
import time
import torch
import pandas as pd

2025-04-10 08:41:59.448302: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744274519.477011   93301 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744274519.486238   93301 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744274519.515865   93301 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744274519.515883   93301 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1744274519.515885   93301 computation_placer.cc:177] computation placer alr

In [2]:
# Load test data
dataset = load_dataset("gigaword", split="test[:100]")  # Limit to 100 for fast eval

In [ ]:
import torch

def empty_GPU_cache():
    """
    Clear GPU memory cache.
    """
    del benchmark.model
    del benchmark.tokenizer
    del benchmark.llm  # If using llama.cpp
    torch.cuda.empty_cache()
    benchmark.close()  # Shutdown NVML

    print("GPU memory released and NVML shutdown complete.")


GPU memory released and NVML shutdown complete.


In [4]:
def sum_prompt(document):
    """
    Summarize the given `document` into a concise headline using a few-shot prompt.
    """
    prompt = (
        "You are a headline generation assistant. Given a news article, produce a concise and informative headline.\n\n"

        "Here are some examples:\n"

        "News: Scientists have discovered a new exoplanet that appears to have water on its surface, raising hopes it may be habitable.\n"
        "Headline: New exoplanet may support life\n\n"

        "News: The stock market experienced a significant downturn today, with major indices falling sharply amid economic uncertainty.\n"
        "Headline: Stock market plunges amid economic fears\n\n"

        f"News: {document}\n"
        "Headline:"
    )

    return prompt


print(sum_prompt(dataset[0]["document"]))

You are a headline generation assistant. Given a news article, produce a concise and informative headline.

Here are some examples:
News: Scientists have discovered a new exoplanet that appears to have water on its surface, raising hopes it may be habitable.
Headline: New exoplanet may support life

News: The stock market experienced a significant downturn today, with major indices falling sharply amid economic uncertainty.
Headline: Stock market plunges amid economic fears

News: japan 's nec corp. and UNK computer corp. of the united states said wednesday they had agreed to join forces in supercomputer sales .
Headline:


### Summarization - Gemma3-12B q4_k_m 

In [5]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "google_gemma-3-12b-it-q4_k_m"

llama_model_path = f"/home/ubuntu/fast_llm_inference/gemma-3-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="summarization",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=20,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

results = benchmark.benchmark(prompts=[sum_prompt(i) for i in dataset["document"]], 
                              references=[i for i in dataset["summary"]])

empty_GPU_cache()

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory released and NVML shutdown complete.


In [7]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_summarization.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for google_gemma-3-12b-it-q4_k_m:
prompt_length                         671.680000
ATL                                     0.104720
GL                                      0.754810
TPS                                     9.670200
SPS                                     1.394400
Memory Usage (MB)                   10005.360000
Model Size (MB)                      7300.575232
Total Energy (Wh)                       0.013258
Energy per Token (J/token)              7.563756
Energy per Sentence (J/sentence)       46.861596
Energy per Second (W)                  63.259000
ROUGE-1                                 0.360688
ROUGE-L                                 0.331437
BERTScore                               0.865379
dtype: float64


### Summarization - Gemma-3-12B q8_0

In [6]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "google_gemma-3-12b-it-q8"

llama_model_path = f"/home/ubuntu/fast_llm_inference/gemma-3-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="summarization",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=20,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

results = benchmark.benchmark(prompts=[sum_prompt(i) for i in dataset["document"]], 
                              references=[i for i in dataset["summary"]])

empty_GPU_cache()

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory released and NVML shutdown complete.


In [8]:
import pandas as pd

results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_summarization.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for google_gemma-3-12b-it-q8:
prompt_length                         671.680000
ATL                                     0.168661
GL                                      1.229040
TPS                                     5.961300
SPS                                     0.856300
Memory Usage (MB)                   15871.480000
Model Size (MB)                     13453.668352
Total Energy (Wh)                       0.024536
Energy per Token (J/token)             13.864794
Energy per Sentence (J/sentence)       86.717467
Energy per Second (W)                  71.866500
ROUGE-1                                 0.350702
ROUGE-L                                 0.321152
BERTScore                               0.863860
dtype: float64


### Summarization - llama-3.1-8B-Instruct-Q4_K_M

In [ ]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "llama-3.1-8B-Instruct-Q4_K_M"

llama_model_path = f"/home/ubuntu/fast_llm_inference/llama-3.1-8B-Instruct-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="summarization",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=20,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

results = benchmark.benchmark(prompts=[sum_prompt(i) for i in dataset["document"]], 
                              references=[i for i in dataset["summary"]])

empty_GPU_cache()

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (8192) -- the full capacity of the model will not be utilized
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory released and NVML shutdown complete.


In [9]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_summarization.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for llama-3.1-8B-Instruct-Q4_K_M:
prompt_length                        671.680000
ATL                                    0.062315
GL                                     0.455436
TPS                                   16.308800
SPS                                    2.218200
Memory Usage (MB)                   7697.880000
Model Size (MB)                     4920.734368
Total Energy (Wh)                      0.008088
Energy per Token (J/token)             4.541464
Energy per Sentence (J/sentence)      28.959549
Energy per Second (W)                 63.964800
ROUGE-1                                0.351636
ROUGE-L                                0.322978
BERTScore                              0.864453
dtype: float64


### Summarization - llama-3.1-8B-Instruct-Q8_0

In [10]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "llama-3.1-8B-Instruct-Q8_0"

llama_model_path = f"/home/ubuntu/fast_llm_inference/llama-3.1-8B-Instruct-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="summarization",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=20,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

results = benchmark.benchmark(prompts=[sum_prompt(i) for i in dataset["document"]], 
                              references=[i for i in dataset["summary"]])

empty_GPU_cache()

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (8192) -- the full capacity of the model will not be utilized
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory released and NVML shutdown complete.


In [11]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_summarization.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for llama-3.1-8B-Instruct-Q8_0:
prompt_length                         671.680000
ATL                                     0.103335
GL                                      0.717430
TPS                                     9.909000
SPS                                     1.407600
Memory Usage (MB)                   11919.860000
Model Size (MB)                      8540.770976
Total Energy (Wh)                       0.012569
Energy per Token (J/token)              7.424708
Energy per Sentence (J/sentence)       45.000434
Energy per Second (W)                  63.087200
ROUGE-1                                 0.344395
ROUGE-L                                 0.317881
BERTScore                               0.864206
dtype: float64


### Summarization - llama-3.1-8B-Instruct-f16

In [12]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "llama-3.1-8B-Instruct-f16"

llama_model_path = f"/home/ubuntu/fast_llm_inference/llama-3.1-8B-Instruct-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="summarization",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=20,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

results = benchmark.benchmark(prompts=[sum_prompt(i) for i in dataset["document"]], 
                              references=[i for i in dataset["summary"]])

empty_GPU_cache()

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (8192) -- the full capacity of the model will not be utilized
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


GPU memory released and NVML shutdown complete.


In [13]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_summarization.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for llama-3.1-8B-Instruct-f16:
prompt_length                         671.680000
ATL                                     0.177835
GL                                      1.236803
TPS                                     5.763100
SPS                                     0.815900
Memory Usage (MB)                   19667.680000
Model Size (MB)                     16068.891296
Total Energy (Wh)                       0.024831
Energy per Token (J/token)             14.631968
Energy per Sentence (J/sentence)       88.945786
Energy per Second (W)                  72.276300
ROUGE-1                                 0.348157
ROUGE-L                                 0.321643
BERTScore                               0.864405
dtype: float64


In [45]:
#find best bertscore

results.loc[results['BERTScore'] > 0.9]

,Prompt Length,Question (Prompt),Generated Answer,Reference,ROUGE_avg,BERTScore,FTL (s),ATL (s),GL (s),TPS (tokens/s),SPS (sentences/s),Memory Usage (MB),Model Size (MB),KV-Cache Size Estimation (MB),Total Energy (Wh),Energy per Token (J/token),Energy per Sentence (J/sentence),Energy per Second (W)
9,499,You are a headline generation assistant. Given...,UNK,UNK,1.000000,1.000000,0.0884,0.0884,1.3255,11.32,0.75,16922.06,16068.891296,853.168704,0.026255,6.301306,94.519587,71.31
29,659,You are a headline generation assistant. Given...,Man sentenced to years in jail for murdering h...,a court here thursday sentenced a ##-year-old ...,0.435897,0.900535,0.0784,0.0784,1.3326,12.76,0.75,16922.06,16068.891296,853.168704,0.026885,5.693390,96.787630,72.63
43,499,You are a headline generation assistant. Given...,UNK,UNK,1.000000,1.000000,0.0896,0.0896,1.3436,11.16,0.74,16922.06,16068.891296,853.168704,0.027342,6.561967,98.429501,73.26
62,635,You are a headline generation assistant. Given...,"India wins toss, chooses to bat in Antigua Test",india won the toss and chose to bat on the ope...,0.371429,0.905336,0.0888,0.0888,1.3316,11.26,0.75,16922.06,16068.891296,853.168704,0.027609,6.626269,99.394037,74.64


In [48]:
#find best ROUGE

results.loc[results['ROUGE_avg'].between(0.4, 1)]

,Prompt Length,Question (Prompt),Generated Answer,Reference,ROUGE_avg,BERTScore,FTL (s),ATL (s),GL (s),TPS (tokens/s),SPS (sentences/s),Memory Usage (MB),Model Size (MB),KV-Cache Size Estimation (MB),Total Energy (Wh),Energy per Token (J/token),Energy per Sentence (J/sentence),Energy per Second (W)
3,610,You are a headline generation assistant. Given...,Factory orders rise in September,factory orders for manufactured goods rose #.#...,0.400000,0.877697,0.0747,0.0747,1.3450,13.38,0.74,16922.06,16068.891296,853.168704,0.026055,5.211036,93.798656,69.74
9,499,You are a headline generation assistant. Given...,UNK,UNK,1.000000,1.000000,0.0884,0.0884,1.3255,11.32,0.75,16922.06,16068.891296,853.168704,0.026255,6.301306,94.519587,71.31
19,635,You are a headline generation assistant. Given...,France names unchanged team for second test,"france , still high after their convincing ##-...",0.400000,0.866549,0.0743,0.0743,1.3371,13.46,0.75,16922.06,16068.891296,853.168704,0.026208,5.241527,94.347494,70.56
29,659,You are a headline generation assistant. Given...,Man sentenced to years in jail for murdering h...,a court here thursday sentenced a ##-year-old ...,0.435897,0.900535,0.0784,0.0784,1.3326,12.76,0.75,16922.06,16068.891296,853.168704,0.026885,5.693390,96.787630,72.63
41,688,You are a headline generation assistant. Given...,"Japan's largest credit union collapses, incurr...","japan 's collapsed kizu credit union , the lar...",0.410256,0.871471,0.0956,0.0956,1.3383,10.46,0.75,16922.06,16068.891296,853.168704,0.027635,7.106145,99.486031,74.34
43,499,You are a headline generation assistant. Given...,UNK,UNK,1.000000,1.000000,0.0896,0.0896,1.3436,11.16,0.74,16922.06,16068.891296,853.168704,0.027342,6.561967,98.429501,73.26
56,664,You are a headline generation assistant. Given...,Ericsson reaches deal to sell relay production...,swedish telecommunications giant ericsson has ...,0.545455,0.885304,0.0832,0.0832,1.3316,12.02,0.75,16922.06,16068.891296,853.168704,0.026225,5.900665,94.410639,70.90
81,693,You are a headline generation assistant. Given...,Prince Philip marks 60 years of marriage to Qu...,britain 's prince philip celebrates his ##th b...,0.409091,0.847985,0.0789,0.0789,1.3418,12.67,0.75,16922.06,16068.891296,853.168704,0.026613,5.635722,95.807276,71.40
95,654,You are a headline generation assistant. Given...,Notre Dame square to be renamed after Pope Joh...,the open square in front of the french capital...,0.487179,0.859504,0.0835,0.0835,1.3364,11.97,0.75,16922.06,16068.891296,853.168704,0.026276,5.912092,94.593466,70.78


### LLaMA 3.1 8B Summarization Benchmark

<small>

#### 📝 Tested Models
| **Model File**                      | **Precision** | **Quantization Scheme** | **Notes**                                     |
|-------------------------------------|---------------|-------------------------|-----------------------------------------------|
| llama-3.1-8B-f16.gguf               | float16       | Full Precision          | Largest model, highest theoretical accuracy, slowest inference |
| llama-3.1-8B-Q8_0.gguf              | 8-bit         | Q8_0                   | Reduced size, good balance of speed and quality |
| llama-3.1-8B-Q4_K_M.gguf            | 4-bit         | Q4_K_M                 | Highly optimized 4-bit quantization, best summarization quality in tests |

---

### 📊 Summarization Benchmark – Quality, Latency, Efficiency

<small>

| **Metric**                           | **llama-3.1-8B-Instruct-f16** | **llama-3.1-8B-Instruct-Q8_0** | **llama-3.1-8B-Instruct-Q4_K_M** | **google_gemma-3-12b-it-Q4_K_M** | **google_gemma-3-12b-it-Q8** |
|--------------------------------------|-------------------------------|--------------------------------|----------------------------------|----------------------------------|------------------------------|
| **Number of Samples**                | 100                           | 100                            | 100                              | 100                              | 100                          |
| **Prompt Length (avg)**              | 671.6800 tokens               | 671.6800 tokens                | 671.6800 tokens                  | 671.6800 tokens                  | 671.6800 tokens              |
| **ROUGE-1**                          | 0.3482                        | 0.3444                         | 0.3516                           | **0.3607**                       | 0.3507                       |
| **ROUGE-L**                          | 0.3216                        | 0.3179                         | 0.3230                           | **0.3314**                       | 0.3212                       |
| **BERTScore**                        | 0.8644                        | 0.8642                         | 0.8645                           | **0.8654**                       | 0.8639                       |
| **ATL (s)**                          | 0.1778                        | 0.1033                         | **0.0623**                       | 0.1047                           | 0.1687                       |
| **GL (s)**                           | 1.2368                        | 0.7174                         | **0.4554**                       | 0.7548                           | 1.2290                       |
| **TPS (tokens/s)**                   | 5.7631                        | 9.9090                         | **16.3088**                      | 9.6702                           | 5.9613                       |
| **SPS (sentences/s)**                | 0.8159                        | 1.4076                         | **2.2182**                       | 1.3944                           | 0.8563                       |
| **Memory Usage (MB)**                | 19667.68 MB                   | 11919.86 MB                    | **7697.88 MB**                   | 10005.36 MB                      | 15871.48 MB                  |
| **Model Size (MB)**                  | 16068.89 MB                   | 8540.77 MB                     | **4920.73 MB**                   | 7300.58 MB                       | 13453.67 MB                  |
| **Total Energy (Wh)**                | 0.024831                      | 0.012569                       | **0.008088**                     | 0.013258                         | 0.024536                     |
| **Energy per Token (J/token)**       | 14.6320                       | 7.4247                         | **4.5415**                       | 7.5638                           | 13.8648                      |
| **Energy per Sentence (J/sentence)** | 88.9458                       | 45.0004                        | **28.9595**                      | 46.8616                          | 86.7175                      |
| **Energy per Second (W)**            | 72.2763 W                     | **63.0872 W**                  | 63.9648 W                        | 63.2590 W                        | 71.8665 W                    |

</small>



---

#### 🔍 Summary of Insights

- **4-bit quantized models (Q4_K_M)** outperformed both 8-bit and float16 variants across nearly all benchmark metrics, including **ROUGE**, **BERTScore**, **inference latency** (FTL/GL), and **energy efficiency**.
- Despite using the lowest numerical precision, **Q4_K_M** achieved the **highest summarization quality**, **fastest generation speed**, and **lowest memory and energy usage**, making it the best overall performer in this benchmark.
- Surprisingly, the more precise **8-bit (Q8_0)** and **float16** models scored lower in **ROUGE** and **BERTScore**, while also being slower and less efficient. Possible explanations include:
  - Differences in **prompt formatting** or few-shot setup
  - Variations in **sampling parameters** (e.g., temperature, top-p)
  - Potential mismatches in **model variants** (e.g., base vs. instruction-tuned)

---

#### ⚡ Why Quantized Models Are Faster

- **Smaller memory footprint**  
  ➤ Less data to move = faster memory access (critical for LLMs).

- **Faster matrix multiplications**  
  ➤ Quantized operations use low-bit integer math, which is much faster on both CPUs and GPUs.

- **Reduced KV cache size**  
  ➤ Speeds up attention mechanisms and reduces memory bandwidth usage during generation.

- **Lower VRAM/RAM pressure**  
  ➤ Enables more efficient layer-wise parallelism and fewer memory stalls.

- **Hardware-accelerated low-bit operations**  
  ➤ Modern CPUs and GPUs (e.g., Apple Silicon, NVIDIA Tensor Cores) are optimized for INT4/INT8 computation.


</small>

### 🧠 Q4_K_M vs Q8_0 Quantization Comparison

<small>

Both **Q4_K_M** and **Q8_0** are quantization methods used to compress model weights for faster inference and lower memory usage.

---

#### Q4_K_M (4-bit Quantization, Optimized)

| Feature          | Description |
|------------------|-------------|
| **Precision**    | 4-bit |
| **Quantization Type** | "K" series, specifically **K_M** (multi-purpose optimized) |
| **Compression**  | Very high (significantly smaller than 8-bit) |
| **Speed**        | Extremely fast, ideal for CPU/GPU |
| **Memory Usage** | Very low (fits on smaller GPUs like 6-8GB VRAM) |
| **Accuracy**     | Preserves high accuracy in **instruction-tuned tasks** like **summarization**, **chat**, and **QA** |
| **Best Use Cases** | Chatbots, summarization, reasoning tasks |
| **Notes**        | Uses **groupwise quantization** and **per-channel scaling** for better accuracy retention despite low precision |

---

#### Q8_0 (8-bit Quantization, General Purpose)

| Feature          | Description |
|------------------|-------------|
| **Precision**    | 8-bit |
| **Quantization Type** | Uniform 8-bit |
| **Compression**  | Moderate (smaller than float16 but larger than 4-bit) |
| **Speed**        | Faster than float16, but slower than Q4_K_M |
| **Memory Usage** | Moderate (needs more VRAM, typically 12GB+) |
| **Accuracy**     | Higher precision retention in general, but not optimized for specific tasks |
| **Best Use Cases** | Complex reasoning, precision-sensitive tasks |
| **Notes**        | General-purpose quantization without task-specific optimizations |

---

#### ✅ Why Q4_K_M Outperformed Q8_0 in Summarization
- **Q4_K_M** is optimized for **task-specific performance**, often giving better results for **instruction-tuned models**, **summarization**, and **chat** tasks.
- **Q8_0** retains more raw precision but isn't tuned for these tasks, leading to lower scores in ROUGE evaluation.
- **Q4_K_M** also runs significantly faster with less resource usage.

---

</small>

### 🧠 How Q4_K_M Is Optimized

<small>

Q4_K_M is part of the advanced **K series** quantization schemes, designed to balance **speed**, **size**, and **accuracy**. It introduces several optimizations to maintain high task performance despite being a 4-bit quantization.

#### Key Optimizations

| Optimization                   | Description |
|--------------------------------|-------------|
| **Groupwise Quantization**     | Weights are divided into small groups (e.g., 32 or 64) and quantized individually, improving precision retention. |
| **Per-Channel Scaling**        | Each group or channel has its own scale factor, ensuring finer control over the quantization process. |
| **Mixed Weight Packing (M)**   | Uses different packing strategies optimized for different layers (e.g., attention vs MLP layers). |
| **Dynamic Zero Points**        | Zero points are dynamically computed within groups, reducing quantization bias. |
| **Efficient SIMD Utilization** | The packed format is optimized for vectorized operations on CPU and GPU, increasing inference speed. |

#### Why It Works Well
- Optimized for **instruction-following**, **summarization**, and **chat** tasks.
- Preserves task-critical accuracy despite aggressive compression.
- Runs **very efficiently** on both CPU and GPU.

#### Q4_K_M vs Q8_0

| Feature            | Q4_K_M                   | Q8_0                 |
|--------------------|--------------------------|----------------------|
| Precision          | 4-bit                    | 8-bit               |
| Compression        | High                     | Medium              |
| Accuracy Retention | High (task-optimized)    | High (general)      |
| Speed              | Very fast                | Fast                |
| Memory Usage       | Very low                 | Medium              |
| Task Tuning        | Summarization, Chat, QA  | General-purpose     |
| Ollama Use         | ✅ Often used (default)  | ❌ Less common       |

</small>

### Benchmarking Q&A

In [1]:
from datasets import load_dataset
import random
import time

# Load SQuAD v2 dataset (validation split)
squad_v2 = load_dataset("squad_v2")

# Set random seed for reproducibility
random.seed(42)

# Convert the validation split to a list and sample 200 random questions
validation_list = list(squad_v2["validation"])
sampled_questions = random.sample(validation_list, 200)

questions_with_answers = [i for i in sampled_questions if len(i['answers']['text']) > 0]

len(questions_with_answers)

101

In [3]:
def qa_prompt(example):
    context = example['context']
    question = example['question']

    prompt_template = (
        "You are a question answering assistant. Given the context, answer the question. "
        "If the answer isn't in the context, respond 'I don't know'.\n\n"

        "Here is an example:\n"
        "Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni)...\n"
        "Question: What is the name of the region the Normans gave their name to?\n"
        "Answer: Normandy\n\n"

        "Context: {context}\n\n"
        "Question: {question}\n\n"
        "Answer:"
    )
    
    return prompt_template.format(context=context, question=question)

### Gemma-3-12B Q&A

In [ ]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "google_gemma-3-12b-it-q4_k_m"

llama_model_path = f"/home/ubuntu/fast_llm_inference/gemma-3-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="qa",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=20,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

results = benchmark.benchmark(prompts=[qa_prompt(i) for i in questions_with_answers], 
                              references=[quest['answers']['text'] for quest in questions_with_answers])

empty_GPU_cache()

In [10]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_Q&A.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for google_gemma-3-12b-it-q4_k_m:
prompt_length                       1248.297030
ATL                                    0.079040
GL                                     0.438619
TPS                                   10.692970
SPS                                    3.764950
Memory Usage (MB)                   8965.881782
Model Size (MB)                     7300.575232
Total Energy (Wh)                      0.008629
Energy per Token (J/token)             8.988606
Energy per Sentence (J/sentence)      23.808748
Energy per Second (W)                 70.761386
exact_match                            0.613861
F1_score                               0.828935
dtype: float64


In [11]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "google_gemma-3-12b-it-q8"

llama_model_path = f"/home/ubuntu/fast_llm_inference/gemma-3-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="qa",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=20,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

results = benchmark.benchmark(prompts=[qa_prompt(i) for i in questions_with_answers], 
                              references=[quest['answers']['text'] for quest in questions_with_answers])

empty_GPU_cache()

llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (131072) -- the full capacity of the model will not be utilized


GPU memory released and NVML shutdown complete.


In [12]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_Q&A.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for google_gemma-3-12b-it-q8:
prompt_length                        1248.297030
ATL                                     0.118337
GL                                      0.612650
TPS                                     7.068416
SPS                                     2.837030
Memory Usage (MB)                   14836.515446
Model Size (MB)                     13453.668352
Total Energy (Wh)                       0.012074
Energy per Token (J/token)             12.978702
Energy per Sentence (J/sentence)       33.038025
Energy per Second (W)                  70.939703
exact_match                             0.663366
F1_score                                0.855050
dtype: float64


### Llama3.1-8B Q&A

In [ ]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "llama-3.1-8B-Instruct-Q4_K_M"

llama_model_path = f"/home/ubuntu/fast_llm_inference/llama-3.1-8B-Instruct-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=10,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

answers = [x['answers']['text'] for x in questions_with_answers]

results = benchmark.benchmark([qa_prompt(i) for i in questions_with_answers], answers, task="qa")

empty_GPU_cache()

llama_init_from_model: n_ctx_per_seq (4096) < n_ctx_train (8192) -- the full capacity of the model will not be utilized


In [ ]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_Q&A.csv", index=False)

averages = results.select_dtypes(include='number').mean()
print(averages)

prompt_length                       1248.297030
ATL                                    0.059390
GL                                     0.414821
TPS                                   18.133267
SPS                                    3.017030
Memory Usage (MB)                   5933.545149
Model Size (MB)                     4920.734368
Total Energy (Wh)                      0.004908
Energy per Token (J/token)             2.533958
Energy per Sentence (J/sentence)      15.793119
Energy per Second (W)                 42.552673
exact_match                            0.663366
F1_score                               0.853918
dtype: float64


### LLaMA 3.1 8B Instruct - Quantization Benchmark (SQuAD v2 Q&A)

<small>

| **Metric**                        | **4-bit (Q4_K_M)**              | **8-bit (Q8_0)**              | **fp16**                      | **GEMMA Q4_K_M**             | **GEMMA Q8**               |
|-----------------------------------|---------------------------------|-------------------------------|-------------------------------|-------------------------------|-----------------------------|
| **Quantization Technique**        | 4-bit Group Quantization (Q4_K_M) | 8-bit Quantization (Q8_0)    | FP16 (Half-Precision Float)   | 4-bit Group Quantization (Q4_K_M) | 8-bit Quantization (Q8) |
| **Number of Examples**            | 101                             | 101                           | 101                           | 101                           | 101                         |
| **Prompt Length (avg)**           | 1248.2970 tokens                | 1248.2970 tokens              | 1248.2970 tokens              | 1248.2970 tokens              | 1248.2970 tokens            |
| **ATL (Average Token Latency)**   | **0.0539 s/token**              | 0.0736 s/token                | 0.1077 s/token                | 0.0790 s/token                | 0.1183 s/token              |
| **GL (Generation Latency)**       | **0.3762 s**                    | 0.5008 s                      | 0.7518 s                      | 0.4386 s                      | 0.6127 s                    |
| **TPS (Tokens/sec)**              | **19.8857**                     | 14.5372                       | 9.7952                        | 10.6930                       | 7.0684                      |
| **SPS (Sentences/sec)**           | 3.3143                          | 2.6915                        | 1.7902                        | **3.7649**                    | 2.8370                      |
| **Memory Usage (MB)**             | **5933.5451 MB**                | 9135.4461 MB                  | 15912.1392 MB                 | 8965.8818 MB                  | 14836.5154 MB               |
| **Model Size (MB)**               | **4920.7344 MB**                | 8540.7710 MB                  | 16068.8913 MB                 | 7300.5752 MB                  | 13453.6684 MB               |
| **Total Energy (Wh)**             | **0.0072 Wh**                   | 0.0095 Wh                     | 0.0147 Wh                     | 0.0086 Wh                     | 0.0121 Wh                   |
| **Energy per Token (J/token)**    | **3.6804**                      | 5.0064                        | 7.5682                        | 8.9886                        | 12.9787                     |
| **Energy per Sentence (J/sentence)** | **22.9896**                   | 29.2293                       | 44.9827                       | 23.8087                       | 33.0380                     |
| **Energy per Second (W)**         | **68.7679 W**                   | 68.5086 W                     | 70.4359 W                     | 70.7614 W                     | 70.9397 W                   |
| **Exact Match (EM)**              | 0.6634                          | **0.6733**                    | 0.6634                        | 0.6139                        | 0.6634                      |
| **F1 Score**                      | 0.8539                          | 0.8498                        | 0.8399                        | 0.8289                        | **0.8551**                  |

</small>

### Text-to-SQL - Quantization Benchmark

In [1]:
from datasets import load_dataset
spider = load_dataset("spider", split="train[:100]")

# Sample
print("Q:", spider[0]["question"])
print("SQL:", spider[0]["query"])

print("Number of samples:", len(spider))

Q: How many heads of the departments are older than 56 ?
SQL: SELECT count(*) FROM head WHERE age  >  56
Number of samples: 100


In [2]:
import json

# Function to extract and format columns grouped by table
def get_table_columns(db_id: str) -> str:

    # Load tables.json
    with open("/home/ubuntu/fast_llm_inference/tables.json", "r") as f:
        tables_json = json.load(f)

    for db in tables_json:
        if db["db_id"] == db_id:
            table_names = db["table_names_original"]
            column_info = db["column_names_original"]
            
            table_columns = {table: [] for table in table_names}
            for table_idx, col_name in column_info:
                if col_name == "*":
                    continue  # skip wildcard
                table = table_names[table_idx]
                table_columns[table].append(col_name)
            
            # Now build the formatted string
            lines = []
            for table, cols in table_columns.items():
                lines.append(f"Table '{table}': columns = {', '.join(cols)}")
            return "\n".join(lines)
    
    return "No schema found for that db_id."

def sql_prompt(dataset, index):
    question = dataset["question"][index]
    db_id = dataset["db_id"][index]
    columns_str = get_table_columns(db_id)

    prompt_template = (
        "You are a SQL query generation assistant. Given a natural language question, generate the corresponding SQL query. Only generate valid SQL statements, no text!\n\n"
        
        "Here is an example:\n\n"

        "Question: How many heads of the departments are older than 56?\n\n"

        "Tables in the database:\n"
        "Table 'department': columns = Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees\n"
        "Table 'head': columns = head_ID, name, born_state, age\n"
        "Table 'management': columns = department_ID, head_ID, temporary_acting\n\n"
        "SQL: SELECT count(*) FROM head WHERE age > 56\n\n"
        
        "Question: {question}\n\n"
        
        "Tables in the database:\n"
        "{column_context}\n\n"
        "SQL:"
    )
    
    return prompt_template.format(
        question=question,
        column_context=columns_str
    )

prompt = sql_prompt(spider, 0)
print(prompt)

You are a SQL query generation assistant. Given a natural language question, generate the corresponding SQL query. Only generate valid SQL statements, no text!

Here is an example:

Question: How many heads of the departments are older than 56?

Tables in the database:
Table 'department': columns = Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees
Table 'head': columns = head_ID, name, born_state, age
Table 'management': columns = department_ID, head_ID, temporary_acting

SQL: SELECT count(*) FROM head WHERE age > 56

Question: How many heads of the departments are older than 56 ?

Tables in the database:
Table 'department': columns = Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees
Table 'head': columns = head_ID, name, born_state, age
Table 'management': columns = department_ID, head_ID, temporary_acting

SQL:


### Text-to-SQL Gemma3 12B

In [ ]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "google_gemma-3-12b-it-q4_k_m"

llama_model_path = f"/home/ubuntu/fast_llm_inference/gemma-3-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="sql",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=70,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

dataset = spider
n = len(dataset['query'])

results = benchmark.benchmark([sql_prompt(dataset, i) for i in range(n)], [dataset['query'][i] for i in range(n)])

empty_GPU_cache()

In [4]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_sql.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for google_gemma-3-12b-it-q4_k_m:
prompt_length                       1206.040000
ATL                                    0.274330
GL                                     2.596460
TPS                                    4.943200
SPS                                    0.784500
Memory Usage (MB)                   8960.040000
Model Size (MB)                     7300.575232
Total Energy (Wh)                      0.051605
Energy per Token (J/token)            19.617788
Energy per Sentence (J/sentence)     147.776944
Energy per Second (W)                 71.542000
AST_equal                              0.440000
Normalized_equal                       0.450000
dtype: float64


In [ ]:
from benchmark.benchmark import ModelBenchmark
import os

model_name = "google_gemma-3-12b-it-q8"

llama_model_path = f"/home/ubuntu/fast_llm_inference/gemma-3-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="sql",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=70,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

dataset = spider
n = len(dataset['query'])

results = benchmark.benchmark([sql_prompt(dataset, i) for i in range(n)], [dataset['query'][i] for i in range(n)])

empty_GPU_cache()

In [4]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_sql.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for google_gemma-3-12b-it-q8:
prompt_length                        1206.040000
ATL                                     0.420400
GL                                      4.020178
TPS                                     3.446400
SPS                                     0.565400
Memory Usage (MB)                   14827.740000
Model Size (MB)                     13453.668352
Total Energy (Wh)                       0.080359
Energy per Token (J/token)             30.256650
Energy per Sentence (J/sentence)      221.414288
Energy per Second (W)                  71.959600
AST_equal                               0.390000
Normalized_equal                        0.390000
dtype: float64


### Text-to-SQL LLama3.1 8B

In [ ]:
from benchmark.benchmark import ModelBenchmark
import os
import torch

model_name = "llama-3.1-8B-Instruct-f16"

llama_model_path = f"/home/ubuntu/fast_llm_inference/llama-3.1-8B-Instruct-gguf/{model_name}.gguf"

benchmark = ModelBenchmark(
    backend="llama.cpp",
    task="sql",
    llama_model_path=llama_model_path,
    llama_gpu_layers=-1,
    max_tokens=70,
    model_size= os.path.getsize(llama_model_path) / 1e6, # in MB
)

dataset = spider
n = len(dataset['query'])

results = benchmark.benchmark([sql_prompt(dataset, i) for i in range(n)], [dataset['query'][i] for i in range(n)])

empty_GPU_cache()

In [4]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_sql.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for llama-3.1-8B-Instruct-f16:
prompt_length                        1206.040000
ATL                                     0.442202
GL                                      4.346079
TPS                                     2.947900
SPS                                     0.370000
Memory Usage (MB)                   15514.060000
Model Size (MB)                     16068.891296
Total Energy (Wh)                       0.086791
Energy per Token (J/token)             31.787949
Energy per Sentence (J/sentence)      278.841249
Energy per Second (W)                  71.892400
AST_equal                               0.330000
Normalized_equal                        0.320000
dtype: float64


In [34]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_sql.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for llama-3.1-8B-Instruct-Q8_0:
prompt_length                       1160.040000
ATL                                    0.291190
GL                                     2.809894
TPS                                    4.425600
SPS                                    0.551400
Memory Usage (MB)                   9124.060000
Model Size (MB)                     8540.770976
Total Energy (Wh)                      0.055664
Energy per Token (J/token)            20.767205
Energy per Sentence (J/sentence)     178.564317
Energy per Second (W)                 71.318700
AST_equal                              0.320000
Normalized_equal                       0.320000
dtype: float64


In [29]:
# Save the results to a CSV file
import pandas as pd
results = pd.DataFrame(results)

results.to_csv(f"llama_cpp_results/{model_name}_sql.csv", index=False)

averages = results.select_dtypes(include='number').mean()

print(f"Averages for {model_name}:")
print(averages)

Averages for llama-3.1-8B-Instruct-Q4_K_M:
prompt_length                       1160.040000
ATL                                    0.140297
GL                                     1.367014
TPS                                    8.920900
SPS                                    0.961000
Memory Usage (MB)                   5922.060000
Model Size (MB)                     4920.734368
Total Energy (Wh)                      0.027230
Energy per Token (J/token)            10.062473
Energy per Sentence (J/sentence)      91.429658
Energy per Second (W)                 71.713700
AST_equal                              0.300000
Normalized_equal                       0.300000
dtype: float64


In [23]:
def check_answer(index):
    print("AST:", results['AST_equal'][index])
    print("Prompt:", results['prompt'][index], "\n\n")
    print("Prediction:", results['generated_answer'][index])
    print("Generated query:", results['reference_answer'][index])

In [27]:
check_answer(91)

AST: 0
Prompt: You are a SQL query generation assistant. Given a natural language question, generate the corresponding SQL query.

Here is an example:

Question: How many heads of the departments are older than 56?

Tables in the database:
Table 'department': columns = Department_ID, Name, Creation, Ranking, Budget_in_Billions, Num_Employees
Table 'head': columns = head_ID, name, born_state, age
Table 'management': columns = department_ID, head_ID, temporary_acting
SQL: SELECT count(*) FROM head WHERE age > 56

Question: Find distinct cities of addresses of people?

Tables in the database:
Table 'Addresses': columns = address_id, line_1, line_2, city, zip_postcode, state_province_county, country
Table 'People': columns = person_id, first_name, middle_name, last_name, cell_mobile_number, email_address, login_name, password
Table 'Students': columns = student_id, student_details
Table 'Courses': columns = course_id, course_name, course_description, other_details
Table 'People_Addresses':

### 🔬 Quality vs Speed vs Efficiency Trade-Off Summary

<small>

| Metric                          | LLaMA f16               | LLaMA Q8_0             | LLaMA Q4_K_M           | GEMMA Q4_K_M           | GEMMA Q8               |
|---------------------------------|--------------------------|------------------------|------------------------|------------------------|------------------------|
| **Prompt Length**              | 1160.04                 | 1160.04               | 1160.04               | 1160.04               | 1206.04               |
| **Avg Token Latency (ATL)**    | 0.485                   | 0.291                 | **0.140**             | 0.287                 | 0.420                 |
| **Generation Latency (GL)**    | 4.633                   | 2.810                 | **1.367**             | 2.708                 | 4.020                 |
| **Tokens/sec (TPS)**           | 2.673                   | 4.426                 | **8.921**             | 4.337                 | 3.446                 |
| **Sentences/sec (SPS)**        | 0.331                   | 0.551                 | **0.961**             | 0.717                 | 0.565                 |
| **Memory Usage (MB)**          | 15878.66                | 9124.06               | **5922.06**           | 8960.04               | 14827.74              |
| **Model Size (MB)**            | 16068.89                | 8540.77               | **4920.73**           | 7300.58               | 13453.67              |
| **Total Energy (Wh)**          | 0.092                   | 0.056                 | **0.027**             | 0.054                 | 0.080                 |
| **Energy per Token (J/token)** | 34.822                  | 20.767                | **10.062**            | 20.640                | 30.257                |
| **Energy per Sentence (J/s)**  | 298.992                 | 178.564               | **91.430**            | 156.619               | 221.414               |
| **Energy per Second (W)**      | 71.862                  | **71.319**            | 71.714                | 71.731                | 71.960                |
| **AST_equal**                  | 0.320                   | 0.320                 | 0.300                 | **0.430**             | 0.390                 |
| **Normalized_equal**           | 0.320                   | 0.320                 | 0.300                 | **0.430**             | 0.390                 |


</small>